In [1]:
from agents import Agent, Runner, SQLiteSession

In [2]:
import os
from openai import OpenAI

In [3]:
from openai import AsyncOpenAI

In [4]:
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, set_tracing_disabled
base_url = 'http://localhost:11434/v1'
api_key = 'ollama'
client  = AsyncOpenAI(base_url=base_url,api_key=api_key )
set_tracing_disabled(disabled=True)

In [5]:
import mlflow
from pathlib import Path
current_dir = Path.cwd()
project_root = current_dir.parent
mlflow_logs = project_root / "mlflow_logs"
mlflow_logs.mkdir(exist_ok=True)
db_path = mlflow_logs / "mlflow.db"

mlflow.set_tracking_uri(f"sqlite:///{db_path.resolve()}")
mlflow.openai.autolog()

In [6]:
@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."

In [7]:
MODEL_NAME = 'qwen3:14b'

In [8]:
async def main(prompt):
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You are a helpful assistant.",
        model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
        tools=[get_weather],
    )

    result = await Runner.run(agent, prompt)
    print(result.final_output)

In [9]:
prompt = "What's the weather in Tokyo?"
await main(prompt)

2025/12/04 19:23:43 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/04 19:23:43 INFO mlflow.store.db.utils: Updating database tables
2025-12-04 19:23:43 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-04 19:23:43 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-12-04 19:23:43 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-04 19:23:43 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


[debug] getting weather for Tokyo
The weather in Tokyo is sunny. 😊


In [10]:
prompt = "What are pros and cons of Generative AI ?"
await main(prompt)

The pros and cons of generative AI can be summarized as follows:

### **Pros**:
1. **Creativity & Efficiency**: Generates unique content (text, images, code) quickly, aiding in tasks like writing, design, and problem-solving.
2. **Scalability**: Automates repetitive tasks, reducing time and cost for businesses (e.g., customer support, marketing).
3. **Personalization**: Creates tailored content for users (e.g., personalized recommendations, adaptive learning materials).
4. **Innovation**: Explores novel ideas, aiding research, art, and scientific simulations.

### **Cons**:
1. **Accuracy & Reliability**: May produce hallucinations or incorrect information, requiring human verification.
2. **Ethical Risks**: Potential for misuse (e.g., deepfakes, misinformation, plagiarism).
3. **Bias**: Reflects biases in training data, leading to unfair or discriminatory outputs.
4. **Job Displacement**: Could disrupt industries reliant on routine tasks (e.g., writing, design).

Would you like further